In [1]:
import torch
import cv2
import open_clip

c:\Users\Juanma\anaconda3\envs\new-tfg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-g-14', pretrained='laion2b_s34b_b88k')
model.to('cuda')

In [3]:
tokenizer = open_clip.get_tokenizer('ViT-g-14')

In [4]:
import csv

CLASES = 'objetos.csv'
IMAGES_DIR_VALIDATION = 'Final_dataset_small\Validation'

promps = [
          'a photo of {}, a type of domestic waste',
          'a picture of {}, a type of domestic waste',
          'a photo of a {}, a type of domestic waste',
          'a picture of a {}, a type of domestic waste',
          'an image of {}, a type of domestic waste',
          'an image of a {}, a type of domestic waste',
          'a bright photo of a {}, a type of domestic waste',
          ]

with open(CLASES, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    clases = list(reader)

eng_sub_clases = [clase[1] for clase in clases]
esp_sub_clases = [clase[0] for clase in clases]
clases = [clase[2] for clase in clases]

In [ ]:
promps2 = [
    ('Plastic container', 'Amarillo'),
    ('Cardboard packaging', 'Azul'),
    ('Glass container', 'Verde'),
    ('Metal container', 'Amarillo'),
    ('Battery', 'Pilas'),
    ('Green dot', 'Punto'),
    ('Electrical appliance', 'RAEE'),
    ('Clothes', 'Ropa'),
    ('Organic waste', 'Marron'),
    ('Rest', 'Gris'),
    ('Medicine/Pharmacy', 'Medicamento'),
]

promps2_text = [ promp[0] for promp in promps2 ]
clases = [ promp[1] for promp in promps2 ]

import os
import matplotlib.pyplot as plt
from PIL import Image

results2 = []

with torch.no_grad(), torch.cuda.amp.autocast():
    for promp in promps:
        print('Promp: {}'.format(promp))
        final_promps = [promp.format(promp2).lower() for promp2 in promps2_text]
        print('Examples: {}'.format(final_promps[:5]))

        final_promps = tokenizer(final_promps)
        class_dir = os.listdir(IMAGES_DIR_VALIDATION)

        text_features = model.encode_text(final_promps.to('cuda'))
        text_features /= text_features.norm(dim=-1, keepdim=True)

        for clase in class_dir:
            ok_class = 0
            total_class = 0

            for i, image_dir in enumerate(os.listdir(os.path.join(IMAGES_DIR_VALIDATION, clase))):
                image_dir = os.path.join(IMAGES_DIR_VALIDATION, clase, image_dir)
                image = Image.open(image_dir)
                image_pre = preprocess(image).unsqueeze(0).to('cuda')
                
                image_features = model.encode_image(image_pre)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                top_probs, top_labels = text_probs.cpu().topk(1)

                if clases[top_labels[0][0]] == clase:
                    ok_class += 1
                else:
                    '''
                    print('Clases posibles: {}'.format(posible_clases))

                    # Visualizamos la imagen
                    plt.title(clases[top_labels[0][0]])
                    plt.imshow(image)
                    plt.axis('off')
                    plt.show()
                    '''
                    pass
                total_class += 1

                if i%100 == 0:
                    print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))

            print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))
            results2.append([promp, clase, ok_class, total_class, ok_class/total_class])

In [ ]:
promps2 = [
    ('yellow waste', 'Amarillo'),
    ('blue waste', 'Azul'),
    ('green waste', 'Verde'),
    ('Battery waste', 'Pilas'),
    ('recycling center waste', 'Punto'),
    ('Electrical waste', 'RAEE'),
    ('Clothes waste', 'Ropa'),
    ('Organic waste', 'Marron'),
    ('gray waste', 'Gris'),
    ('medicine waste', 'Medicamento'),
]

promps2_text = [ promp[0] for promp in promps2 ]
clases = [ promp[1] for promp in promps2 ]

import os
import matplotlib.pyplot as plt
from PIL import Image

results2 = []

with torch.no_grad(), torch.cuda.amp.autocast():
    for promp in promps:
        print('Promp: {}'.format(promp))
        final_promps = [promp.format(promp2).lower() for promp2 in promps2_text]
        
        print('Examples: {}'.format(final_promps[:5]))

        final_promps = tokenizer(final_promps)
        class_dir = os.listdir(IMAGES_DIR_VALIDATION)

        text_features = model.encode_text(final_promps.to('cuda'))
        text_features /= text_features.norm(dim=-1, keepdim=True)

        for clase in class_dir:
            ok_class = 0
            total_class = 0

            for i, image_dir in enumerate(os.listdir(os.path.join(IMAGES_DIR_VALIDATION, clase))):
                image_dir = os.path.join(IMAGES_DIR_VALIDATION, clase, image_dir)
                image = Image.open(image_dir)
                image_pre = preprocess(image).unsqueeze(0).to('cuda')
                
                image_features = model.encode_image(image_pre)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                top_probs, top_labels = text_probs.cpu().topk(1)

                if clases[top_labels[0][0]] == clase:
                    ok_class += 1
                else:
                    '''
                    print('Clases posibles: {}'.format(posible_clases))

                    # Visualizamos la imagen
                    plt.title(clases[top_labels[0][0]])
                    plt.imshow(image)
                    plt.axis('off')
                    plt.show()
                    '''
                    pass
                total_class += 1

                if i%100 == 0:
                    print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))

            print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))
            results2.append([promp, clase, ok_class, total_class, ok_class/total_class])

In [10]:
print(clases)

['Punto', 'Resta', 'Amarillo', 'RAEE', 'Punto', 'RAEE', 'RAEE', 'Pilas', 'Resta', 'Resta', 'Amarillo', 'Pilas', 'Amarillo', 'RAEE', 'Resta', 'Punto', 'RAEE', 'RAEE', 'Verde', 'Punto', 'Resta', 'Marron', 'Marron', 'Amarillo', 'Resta', 'Punto', 'Punto', 'Resta', 'RAEE', 'Verde', 'RAEE', 'RAEE', 'RAEE', 'Punto', 'Amarillo', 'Punto', 'Resta', 'Marron', 'Punto', 'Resta', 'Resta', 'Resta', 'RAEE', 'RAEE', 'Amarillo', 'Azul', 'Amarillo', 'RAEE', 'Medicamento', 'Punto', 'Resta', 'Punto', 'Resta', 'Marron', 'Amarillo', 'RAEE', 'Punto', 'Verde', 'RAEE', 'Azul', 'RAEE', 'Azul', 'Medicamento', 'Amarillo', 'Resta', 'RAEE', 'Marron', 'RAEE', 'Pilas', 'Amarillo', 'Medicamento', 'Punto', 'Azul', 'Resta', 'Medicamento', 'RAEE', 'Marron', 'RAEE', 'RAEE', 'Verde', 'Resta', 'Azul', 'Punto', 'Resta', 'RAEE', 'Resta', 'Punto', 'Marron', 'Resta', 'RAEE', 'Ropa', 'RAEE', 'Resta', 'Resta', 'Amarillo', 'Punto', 'RAEE', 'Verde', 'Resta', 'Amarillo', 'Punto', 'Resta', 'Resta', 'Amarillo', 'Marron', 'Punto', 'Azul

In [5]:
import os
import matplotlib.pyplot as plt
from PIL import Image

results = []

with torch.no_grad(), torch.cuda.amp.autocast():
    for promp in promps:
        print('Promp: {}'.format(promp))
        final_promps = [promp.format(promp2).lower() for promp2 in eng_sub_clases]
        print('Examples: {}'.format(final_promps[:5]))

        final_promps = tokenizer(final_promps)
        class_dir = os.listdir(IMAGES_DIR_VALIDATION)

        print('Calculating text features')
        text_features = model.encode_text(final_promps.to('cuda'))
        text_features /= text_features.norm(dim=-1, keepdim=True)

        print('Calculating image features')

        for clase in class_dir:
            ok_class = 0
            total_class = 0

            for i, image_dir in enumerate(os.listdir(os.path.join(IMAGES_DIR_VALIDATION, clase))):
                image_dir = os.path.join(IMAGES_DIR_VALIDATION, clase, image_dir)
                image = Image.open(image_dir)
                image_pre = preprocess(image).unsqueeze(0).to('cuda')
                
                image_features = model.encode_image(image_pre)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                top_probs, top_labels = text_probs.cpu().topk(1)

                #posible_clases = [esp_sub_clases[top_label] for top_label in top_labels[0]]

                if clases[top_labels[0][0]] == clase:
                    ok_class += 1
                else:
                    '''
                    print('Clases posibles: {}'.format(posible_clases))

                    # Visualizamos la imagen
                    plt.title(clases[top_labels[0][0]])
                    plt.imshow(image)
                    plt.axis('off')
                    plt.show()
                    '''
                    pass
                total_class += 1

                if i%100 == 0:
                    print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))

            print('Imagen: {}, ok: {}, total: {}, acc: {}'.format(i, ok_class, total_class, ok_class/total_class))
            results.append([promp, clase, ok_class, total_class, ok_class/total_class])

Promp: a photo of {}, a type of domestic waste
Examples: ['a photo of ceramic vase, a type of domestic waste', 'a photo of plastic glass, a type of domestic waste', 'a photo of bottle caps, a type of domestic waste', 'a photo of blender, a type of domestic waste', 'a photo of varnish, a type of domestic waste']
Calculating text features
Calculating image features
Imagen: 0, ok: 1, total: 1, acc: 1.0
Imagen: 100, ok: 92, total: 101, acc: 0.9108910891089109
Imagen: 200, ok: 176, total: 201, acc: 0.8756218905472637
Imagen: 300, ok: 229, total: 301, acc: 0.760797342192691
Imagen: 400, ok: 319, total: 401, acc: 0.7955112219451371
Imagen: 499, ok: 383, total: 500, acc: 0.766
Imagen: 0, ok: 0, total: 1, acc: 0.0
Imagen: 100, ok: 85, total: 101, acc: 0.8415841584158416
Imagen: 200, ok: 166, total: 201, acc: 0.8258706467661692
Imagen: 300, ok: 221, total: 301, acc: 0.7342192691029901
Imagen: 400, ok: 277, total: 401, acc: 0.6907730673316709
Imagen: 499, ok: 360, total: 500, acc: 0.72
Imagen: 0,

In [6]:
for result in results:
    print(result[0], result[1], result[4], sep='-')

a photo of {}, a type of domestic waste-Amarillo-0.766
a photo of {}, a type of domestic waste-Azul-0.72
a photo of {}, a type of domestic waste-Marron-0.686
a photo of {}, a type of domestic waste-Medicamento-0.665
a photo of {}, a type of domestic waste-Pilas-0.92
a photo of {}, a type of domestic waste-Punto-0.798
a photo of {}, a type of domestic waste-RAEE-0.954
a photo of {}, a type of domestic waste-Resta-0.688
a photo of {}, a type of domestic waste-Ropa-0.858
a photo of {}, a type of domestic waste-Verde-0.74
a picture of {}, a type of domestic waste-Amarillo-0.758
a picture of {}, a type of domestic waste-Azul-0.714
a picture of {}, a type of domestic waste-Marron-0.698
a picture of {}, a type of domestic waste-Medicamento-0.7125
a picture of {}, a type of domestic waste-Pilas-0.94
a picture of {}, a type of domestic waste-Punto-0.802
a picture of {}, a type of domestic waste-RAEE-0.948
a picture of {}, a type of domestic waste-Resta-0.644
a picture of {}, a type of domestic 

In [7]:
# Calculamos la media de los resultados
results_mean = []
for i in range(0, len(results), 10):
    results_mean.append([results[i][0], sum([result[2] for result in results[i:i+10]])/10, sum([result[3] for result in results[i:i+10]])/10, sum([result[4] for result in results[i:i+10]])/10])

print('Media de los resultados')
for result in results_mean:
    print(result)

Media de los resultados
['a photo of {}, a type of domestic waste', 364.7, 470.0, 0.7794999999999999]
['a picture of {}, a type of domestic waste', 365.8, 470.0, 0.78345]
['a photo of a {}, a type of domestic waste', 368.9, 470.0, 0.7883666666666665]
['a picture of a {}, a type of domestic waste', 368.9, 470.0, 0.7889833333333331]
['an image of {}, a type of domestic waste', 364.8, 470.0, 0.7777333333333334]
['an image of a {}, a type of domestic waste', 366.3, 470.0, 0.7820833333333332]
['a bright photo of a {}, a type of domestic waste', 364.7, 470.0, 0.77885]
